In [181]:
import pandas as pd
import openpyxl 
from datetime import datetime 
from scipy.stats import kurtosis
import numpy as np 
import pylab as p
import os
from pathlib import Path
from openpyxl import load_workbook

In [182]:
#get the sorted fin file
file_name_str = r"..\Sray\fin concate esg\AU_bloom_fin_esg.xlsx";
# file_name_str = r"..\Sray\fin concate esg\CN_bloom_fin_esg.xlsx";
# file_name_str = r"..\Sray\fin concate esg\HK_bloom_fin_esg.xlsx";
# file_name_str = r"..\Sray\fin concate esg\MY_bloom_fin_esg.xlsx";
# file_name_str = r"..\Sray\fin concate esg\SG_bloom_fin_esg.xlsx";
file_name = Path(file_name_str).stem
fin_file = pd.ExcelFile(file_name_str)

findata_sheet = pd.read_excel(fin_file, sheet_name="fin_esg", index_col=0, header=0)

fin_data = pd.DataFrame(findata_sheet)

fin_data.head() 


,Year,Dates,Tkr & Exch,Name,Sec_no,PX_LAST,HISTORICAL_MARKET_CAP,TRAIL_12M_SALES_PER_SH,IS_EPS,CF_FREE_CASH_FLOW,PE_RATIO,RETURN_COM_EQY,RETURN_ON_ASSET,VOLATILITY_180D,VOLATILITY_360D,BETA_RAW_OVERRIDABLE,esg,g,s,e
firm no,,,,,,,,,,,,,,,,,,,,
1,2017,2017-12-31,BKW AU Equity,BRICKWORKS LTD,2.0,14.21,2106.1474,5.4231,0.6500,48.453,11.8356,9.1476,6.6192,19.025,19.277,1.4001,25.570868,63.576160,11.396614,1.600725
1,2017,2017-12-29,BKW AU Equity,BRICKWORKS LTD,2.0,14.87,1956.7257,5.6469,0.5506,40.789,11.9092,9.7842,7.1112,19.818,19.340,1.1800,25.570868,63.576160,11.396614,1.600725
1,2018,2018-12-31,BKW AU Equity,BRICKWORKS LTD,2.0,18.38,2446.5306,5.5502,0.7660,47.128,11.9589,9.3071,6.6675,22.331,20.029,0.9532,27.462027,63.576160,12.031439,6.644518
1,2019,2019-12-31,BKW AU Equity,BRICKWORKS LTD,2.0,13.21,2966.1299,6.1702,0.3880,-75.080,17.3853,4.5509,3.1119,33.956,27.923,1.1518,34.825470,69.837448,23.579203,10.933253
1,2020,2020-12-31,BKW AU Equity,BRICKWORKS LTD,2.0,18.68,2804.7458,6.1899,0.4730,14.762,8.7052,13.6242,8.8623,25.609,32.048,1.0724,36.636154,69.837448,23.579203,16.369677


In [183]:
print (fin_data.columns)

Index(['Year', 'Dates', 'Tkr & Exch', 'Name', 'Sec_no', 'PX_LAST',
       'HISTORICAL_MARKET_CAP', 'TRAIL_12M_SALES_PER_SH', 'IS_EPS',
       'CF_FREE_CASH_FLOW', 'PE_RATIO', 'RETURN_COM_EQY', 'RETURN_ON_ASSET',
       'VOLATILITY_180D', 'VOLATILITY_360D', 'BETA_RAW_OVERRIDABLE', 'esg',
       'g', 's', 'e'],
      dtype='object')


In [184]:
fin_data ['es'] = fin_data['e'] * fin_data['s']
fin_data ['eg'] = fin_data['e'] * fin_data['g']
fin_data ['sg'] = fin_data['s'] * fin_data['g']
fin_data['esg_cubic'] = fin_data['esg']**3

In [185]:
fin_data = fin_data.rename({'Name': 'firm_name',
                            'Tkr & Exch': 'tkr',
                            'Sec_no': 'sec_no',
                            'Dates': 'date',
                            'Year':'Year'.lower(),
                            'PE_RATIO': 'pe',
                            'BETA_RAW_OVERRIDABLE': 'beta',
                            'CF_FREE_CASH_FLOW': 'fcf',
                            'RETURN_ON_ASSET': 'roa',
                            'PX_LAST': 'px', 
                            'HISTORICAL_MARKET_CAP': 'HISTORICAL_MARKET_CAP'.lower(), 
                            'TRAIL_12M_SALES_PER_SH': 'TRAIL_12M_SALES_PER_SH'.lower(), 
                            'IS_EPS': 'IS_EPS'.lower(),
                            'RETURN_COM_EQY': 'RETURN_COM_EQY'.lower(),
                            'VOLATILITY_360D':'VOLATILITY_360D'.lower()}, axis=1)
fin_data.head()

,year,date,tkr,firm_name,sec_no,px,historical_market_cap,trail_12m_sales_per_sh,is_eps,fcf,...,volatility_360d,beta,esg,g,s,e,es,eg,sg,esg_cubic
firm no,,,,,,,,,,,,,,,,,,,,,
1,2017,2017-12-31,BKW AU Equity,BRICKWORKS LTD,2.0,14.21,2106.1474,5.4231,0.6500,48.453,...,19.277,1.4001,25.570868,63.576160,11.396614,1.600725,18.242846,101.767953,724.552965,16720.004405
1,2017,2017-12-29,BKW AU Equity,BRICKWORKS LTD,2.0,14.87,1956.7257,5.6469,0.5506,40.789,...,19.340,1.1800,25.570868,63.576160,11.396614,1.600725,18.242846,101.767953,724.552965,16720.004405
1,2018,2018-12-31,BKW AU Equity,BRICKWORKS LTD,2.0,18.38,2446.5306,5.5502,0.7660,47.128,...,20.029,0.9532,27.462027,63.576160,12.031439,6.644518,79.943111,422.432936,764.912685,20710.841748
1,2019,2019-12-31,BKW AU Equity,BRICKWORKS LTD,2.0,13.21,2966.1299,6.1702,0.3880,-75.080,...,27.923,1.1518,34.825470,69.837448,23.579203,10.933253,257.797395,763.550509,1646.711342,42236.795203
1,2020,2020-12-31,BKW AU Equity,BRICKWORKS LTD,2.0,18.68,2804.7458,6.1899,0.4730,14.762,...,32.048,1.0724,36.636154,69.837448,23.579203,16.369677,385.983922,1143.216440,1646.711342,49173.331629


In [186]:
fin_data['date'] = fin_data['date'].astype('datetime64')
fin_data.dtypes

year                               int64
date                      datetime64[ns]
tkr                               object
firm_name                         object
sec_no                           float64
px                               float64
historical_market_cap            float64
trail_12m_sales_per_sh           float64
is_eps                           float64
fcf                              float64
pe                               float64
return_com_eqy                   float64
roa                              float64
VOLATILITY_180D                  float64
volatility_360d                  float64
beta                             float64
esg                              float64
g                                float64
s                                float64
e                                float64
es                               float64
eg                               float64
sg                               float64
esg_cubic                        float64
dtype: object

In [187]:
sectors = []
sectors = (fin_data['sec_no'].drop_duplicates()).to_list()
print(sectors)
len(sectors)

[2.0, 4.0, nan, 9.0, 5.0, 6.0, 8.0, 7.0, 11.0, 1.0, 3.0, 10.0, 12.0]


13

In [188]:
#initial_year=2017
#stop_year=2020
#start_date = '01-'+ str(initial_year) 
#end_date = '12-'+ str(initial_year) 
#print(start_date)
#print(end_date)
    

#temp_dataframe = fin_data.loc[(fin_data['sec_no']==1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
#temp_dataframe

## **Add new indicators** 

In [189]:
# companies_sec = []
# initial_year = 2017
# stop_year = 2023

# for year in range(initial_year, stop_year+1, 1):

#     start_date = '01-' + str(year)
#     if (year < 2021):
#         end_date = '12-' + str(year)
#     elif (year == 2021):
#         end_date = '02-' + str(year)

#     for month in range(1, 13):
#         m = str(month)
#         if (month < 10):
#             start_date = '0' + m + '-' + str(year)
#         else:
#             start_date = m + '-' + str(year)

#         for k in range(len(sectors)):

#             # temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
#             temp_dataframe = fin_data.loc[(
#                 fin_data['sec_no'] == k+1) & (fin_data['date'] == start_date)].copy()
#         print(temp_dataframe)


In [190]:
#seperate each of the companies and store into list companies_financial=[]

# temp_dataframe2 = temp_dataframe.loc[fin_data['date']== (str(m)+ '-' + str(year)) ].copy()

#initialize counter to track the number of company
companies_sec=[]
initial_year=2017
stop_year=2023

for year in range(initial_year,stop_year+1,1):    

    # start_date = '01-'+ str(year) 
    # if (year <2021):
    #     end_date = '12-'+ str(year) 
    # elif (year==2021):    
    #     end_date = '02-'+ str(year) 

    # for month in range (1,13):
    #     m=str(month)
    #     if (month<10):
    #         start_date = '0'+ m + '-' + str(year) 
    #     else:
    #         start_date = m + '-' + str(year)  

    
        for k in range (len(sectors)):
        
            # temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
            temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['year']>=initial_year& (fin_data['year']<=stop_year))].copy()
            # temp_dataframe = fin_data.copy()

            
            # temp_dataframe['ep_sec_mean']= temp_dataframe['ep'].mean()

            # temp_dataframe['ep_sec_sd']= temp_dataframe['ep'].std()

            # temp_dataframe['sp_sec_mean'] = temp_dataframe['sp'].mean()

            # temp_dataframe['sp_sec_sd'] = temp_dataframe['sp'].std()

            # temp_dataframe['cfp_sec_mean'] = temp_dataframe['cfp'].mean()

            # temp_dataframe['cfp_sec_sd'] = temp_dataframe['cfp'].std()

            temp_dataframe['roa_sec_mean'] = temp_dataframe['roa'].mean()

            temp_dataframe['roa_sec_sd'] = temp_dataframe['roa'].std()

            companies_sec.append(temp_dataframe)



In [191]:
#combine all the data 
new_companies_data = pd.concat(companies_sec)
new_companies_data

,year,date,tkr,firm_name,sec_no,px,historical_market_cap,trail_12m_sales_per_sh,is_eps,fcf,...,esg,g,s,e,es,eg,sg,esg_cubic,roa_sec_mean,roa_sec_sd
firm no,,,,,,,,,,,,,,,,,,,,,
27,2017,2017-12-31,COE AU Equity,COOPER ENERGY LTD,1.0,0.3044,520.3507,0.0560,0.0131,-72.337,...,30.409416,76.098740,14.963724,0.000000,0.000000,0.000000,1138.720547,28120.578310,-7.46632,26.742251
27,2017,2017-12-29,COE AU Equity,COOPER ENERGY LTD,1.0,0.3044,433.4094,0.0450,-0.0054,-84.163,...,30.409416,76.098740,14.963724,0.000000,0.000000,0.000000,1138.720547,28120.578310,-7.46632,26.742251
27,2018,2018-12-31,COE AU Equity,COOPER ENERGY LTD,1.0,0.4167,721.5902,0.0421,-0.0075,-121.782,...,33.286388,76.098740,19.347036,4.258532,82.389974,324.068921,1472.285083,36880.774196,-7.46632,26.742251
27,2019,2019-12-31,COE AU Equity,COOPER ENERGY LTD,1.0,0.5666,984.1214,0.0461,0.0037,-19.608,...,34.413036,76.098740,19.981861,7.006947,140.011843,533.219838,1520.594446,40753.881613,-7.46632,26.742251
27,2020,2020-12-31,COE AU Equity,COOPER ENERGY LTD,1.0,0.3652,636.1001,0.0496,-0.0131,-12.489,...,35.358616,76.098740,21.553808,8.275445,178.367354,629.750933,1640.217639,44206.463023,-7.46632,26.742251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2019,2019-12-31,SCW AU Equity,SCENTRE GROUP TRUST 1,12.0,NaN,NaN,0.2989,0.0313,-1104.300,...,57.106930,76.219147,38.029022,56.991844,2167.344108,4343.869734,2898.539624,186237.200989,2.65530,6.840744
184,2020,2020-12-31,SCW AU Equity,SCENTRE GROUP TRUST 1,12.0,NaN,NaN,0.4167,-0.0229,473.800,...,59.279751,76.219147,44.226120,57.324070,2535.221197,4369.191701,3370.877129,208314.311861,2.65530,6.840744
184,2021,2021-12-31,SCW AU Equity,SCENTRE GROUP TRUST 1,12.0,NaN,NaN,0.4400,0.0940,422.900,...,58.213459,76.219147,40.689240,57.656296,2345.990828,4394.513668,3101.299116,197274.166371,2.65530,6.840744


In [192]:
new_companies_data.columns

Index(['year', 'date', 'tkr', 'firm_name', 'sec_no', 'px',
       'historical_market_cap', 'trail_12m_sales_per_sh', 'is_eps', 'fcf',
       'pe', 'return_com_eqy', 'roa', 'VOLATILITY_180D', 'volatility_360d',
       'beta', 'esg', 'g', 's', 'e', 'es', 'eg', 'sg', 'esg_cubic',
       'roa_sec_mean', 'roa_sec_sd'],
      dtype='object')

In [193]:
# try:
#     new_companies_data['ep_z'] = new_companies_data.apply(
#         lambda row: (row.ep - row.ep_sec_mean)/row.ep_sec_sd, axis=1)
# except ZeroDivisionError:
#     new_companies_data['ep_z'] = np.nan

# try:
#     new_companies_data['sp_z'] = new_companies_data.apply(
#         lambda row: (row.sp - row.sp_sec_mean)/row.sp_sec_sd, axis=1)
# except ZeroDivisionError:
#     new_companies_data['sp_z'] = np.nan

# try:
#     new_companies_data['cfp_z'] = new_companies_data.apply(
#         lambda row: (row.cfp - row.cfp_sec_mean)/row.cfp_sec_sd, axis=1)
# except ZeroDivisionError:
#     new_companies_data['cfp_z'] = np.nan


#read the column FROM ep_z to cfp_z
# col = new_companies_data.loc[:, "ep_z":"cfp_z"]
# new_companies_data['value'] = col.mean(axis=1)

try:
    new_companies_data['roa_z'] = new_companies_data.apply(
        lambda row: (row.roa - row.roa_sec_mean)/row.roa_sec_sd, axis=1)
except ZeroDivisionError:
    new_companies_data['roa_z'] = np.nan

# try:
#     new_companies_data['esg_value'] = new_companies_data.apply(
#         lambda row: (row.esg_sray + row.value)/2, axis=1)
# except ZeroDivisionError:
#     new_companies_data['esg_value'] = np.nan


In [194]:
new_companies_data.columns

Index(['year', 'date', 'tkr', 'firm_name', 'sec_no', 'px',
       'historical_market_cap', 'trail_12m_sales_per_sh', 'is_eps', 'fcf',
       'pe', 'return_com_eqy', 'roa', 'VOLATILITY_180D', 'volatility_360d',
       'beta', 'esg', 'g', 's', 'e', 'es', 'eg', 'sg', 'esg_cubic',
       'roa_sec_mean', 'roa_sec_sd', 'roa_z'],
      dtype='object')

In [195]:

#sorting the company
new_companies_data['date'] = pd.to_datetime(
    new_companies_data['date'], format='%m%Y', errors='coerce').dt.to_period('m')

new_companies_data = new_companies_data.sort_values(
    by=["sec_no", "firm_name", "date"])

#arrange according to the Portfolio-no normalization
new_companies_data = new_companies_data[['firm_name', 'tkr', 'sec_no', 'year',
                                            'date', 'esg', 'g', 's', 'e', 'es', 'eg', 'sg', 'esg_cubic', 
                                            'px','historical_market_cap', 'trail_12m_sales_per_sh', 
                                            'is_eps', 'fcf','pe', 'return_com_eqy', 'roa', 'VOLATILITY_180D', 
                                            'volatility_360d','beta']]

new_companies_data


,firm_name,tkr,sec_no,year,date,esg,g,s,e,es,...,historical_market_cap,trail_12m_sales_per_sh,is_eps,fcf,pe,return_com_eqy,roa,VOLATILITY_180D,volatility_360d,beta
firm no,,,,,,,,,,,,,,,,,,,,,
64,BEACH ENERGY LTD,BPT AU Equity,1.0,2017,2017-12,63.715923,78.597229,56.559856,55.934765,3163.662269,...,2833.8138,0.3524,0.0463,34.6,6.8279,26.2746,20.0068,44.217,44.381,1.3995
64,BEACH ENERGY LTD,BPT AU Equity,1.0,2017,2017-12,63.715923,78.597229,56.559856,55.934765,3163.662269,...,2833.8138,0.3524,0.0463,34.6,6.5391,26.2746,20.0068,42.933,44.783,1.3483
64,BEACH ENERGY LTD,BPT AU Equity,1.0,2017,2017-12,63.715923,78.597229,56.559856,55.934765,3163.662269,...,2833.8138,0.3524,0.0463,34.6,6.8279,26.2746,20.0068,44.217,44.381,1.3995
64,BEACH ENERGY LTD,BPT AU Equity,1.0,2017,2017-12,63.715923,78.597229,56.559856,55.934765,3163.662269,...,2833.8138,0.3524,0.0463,34.6,6.5391,26.2746,20.0068,42.933,44.783,1.3483
64,BEACH ENERGY LTD,BPT AU Equity,1.0,2017,2017-12,63.715923,78.597229,56.559856,55.934765,3163.662269,...,2833.8138,0.3524,0.0463,34.6,6.8279,26.2746,20.0068,44.217,44.381,1.3995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,SCENTRE GROUP TRUST 1,SCW AU Equity,12.0,2023,2023-01,NaN,NaN,NaN,NaN,NaN,...,NaN,0.4583,0.0926,555.2,NaN,5.1034,2.6426,NaN,NaN,NaN
184,SCENTRE GROUP TRUST 1,SCW AU Equity,12.0,2023,2023-01,NaN,NaN,NaN,NaN,NaN,...,NaN,0.4583,0.0926,555.2,NaN,5.1034,2.6426,NaN,NaN,NaN
184,SCENTRE GROUP TRUST 1,SCW AU Equity,12.0,2023,2023-01,NaN,NaN,NaN,NaN,NaN,...,NaN,0.4583,0.0926,555.2,NaN,5.1034,2.6426,NaN,NaN,NaN


In [196]:
# new_companies_data.to_excel("SG_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
describe_df_path = ""

if file_name[:2].lower() == 'AU'.lower():
    new_companies_data.to_excel( "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
    describe_df_path = "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Descriptive Statistics.xlsx"

elif file_name[:2].lower() == 'CN'.lower():
    new_companies_data.to_excel( "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
    describe_df_path = "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Descriptive Statistics.xlsx"

elif file_name[:2].lower() == 'HK'.lower():
    new_companies_data.to_excel( "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
    describe_df_path = "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Descriptive Statistics.xlsx"

elif file_name[:2].lower() == 'MY'.lower():
    new_companies_data.to_excel( "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
    describe_df_path = "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Descriptive Statistics.xlsx"

elif file_name[:2].lower() == 'SG'.lower():
    new_companies_data.to_excel( "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
    describe_df_path = "..\Sray\Analysis\\" + file_name[:2] + "\\" + file_name[:2] +"_Descriptive Statistics.xlsx"


## **Descriptive Section**

In [197]:
#Descriptive statistics for e, s, g, esg, ret, value_z, roa_z, vol_90d, vol_180d
describe_df= new_companies_data.describe(include="all")
print(describe_df)

describe_df.to_excel( describe_df_path, sheet_name="main")


               firm_name            tkr        sec_no          year     date  \
count              18704          18704  18704.000000  18704.000000    18704   
unique               334            334           NaN           NaN        7   
top     BEACH ENERGY LTD  BPT AU Equity           NaN           NaN  2017-12   
freq                  56             56           NaN           NaN     4676   
mean                 NaN            NaN      5.077844   2019.625000      NaN   
std                  NaN            NaN      3.077903      2.117691      NaN   
min                  NaN            NaN      1.000000   2017.000000      NaN   
25%                  NaN            NaN      2.000000   2017.750000      NaN   
50%                  NaN            NaN      4.000000   2019.500000      NaN   
75%                  NaN            NaN      7.000000   2021.250000      NaN   
max                  NaN            NaN     12.000000   2023.000000      NaN   

                 esg             g     

## **Total number of companies in each sector**

In [198]:
df_sec_company_count = pd.DataFrame(new_companies_data.groupby(['year','sec_no' ])['tkr'].nunique())
df_sec_company_count.head()

tkr
year sec_no     
2017 1.0      21
     2.0      77
     3.0      35
     4.0      41
     5.0      21

In [199]:
book = load_workbook(describe_df_path)
writer = pd.ExcelWriter(describe_df_path, engine = 'openpyxl')
writer.book = book
df_sec_company_count.to_excel(writer, sheet_name='sec_no_count')
writer.close()

## **Kurtosis**

In [200]:
new_companies_data.drop(['date', 'firm_name','tkr','sec_no' ], axis=1, inplace=True)

In [201]:
new_companies_data['px'] = new_companies_data['px'] .astype(float)
new_companies_data.dtypes

year                        int64
esg                       float64
g                         float64
s                         float64
e                         float64
es                        float64
eg                        float64
sg                        float64
esg_cubic                 float64
px                        float64
historical_market_cap     float64
trail_12m_sales_per_sh    float64
is_eps                    float64
fcf                       float64
pe                        float64
return_com_eqy            float64
roa                       float64
VOLATILITY_180D           float64
volatility_360d           float64
beta                      float64
dtype: object

In [163]:
book = load_workbook(describe_df_path)
writer = pd.ExcelWriter(describe_df_path, engine = 'openpyxl')
writer.book = book
('Kurtosis for normal distribution :', kurtosis(new_companies_data, fisher = True)).to_excel(writer, sheet_name='kurtosis')
writer.close()


Kurtosis for normal distribution : float64
